This notebook uses all of our functions to read all input files and:

- Parse file into networkx graph

- Run each of our algorithms on the graph to get desired output

- Saves into some file

In [1]:
# imports
import glob
import pandas as pd
import sys
import time
sys.path.append('../py_files')

from input_parsing import *
from none import *
from some import *
from many import *
from few import *
from alternate import *

In [2]:
# run through inputs and get results
# currently takes ~15mins to run everything
data_dict = dict()
data_dict['Instance name'] = []
data_dict['n'] = []
data_dict['N'] = []
data_dict['S'] = []
data_dict['M'] = []
data_dict['F'] = []
data_dict['A'] = []

for f in glob.glob('../data/*.txt'):
    data_dict['Instance name'].append(f.split("/")[-1][:-4])

    # Parsing network into networkx graph
    G, s, t = create_network(f)
    
    # number of nodes
    data_dict['n'].append(G.number_of_nodes())

    # NONE
    data_dict['N'].append(solve_none(G,s,t))

    # SOME
    data_dict['S'].append(solve_some(G,s,t))

    # MANY
    data_dict['M'].append(solve_many(G, s, t))

    # FEW
    data_dict['F'].append(solve_few(G, s, t))

    # ALTERNATE
    data_dict['A'].append(solve_alternate(G, s, t))

In [3]:
# convert to pandas DataFrame
df = pd.DataFrame.from_dict(data_dict)
df


,Instance name,n,N,S,M,F,A
0,rusty-2-2000,2000,5,True,-,0,False
1,wall-p-3,20,1,True,-,0,False
2,ski-level20-3,254,-1,-,-1,-1,False
3,gnm-1000-2000-1,1000,-1,False,-,2,False
4,gnm-3000-6000-0,3000,6,True,-,0,False
...,...,...,...,...,...,...,...
149,G-ex,8,3,True,-,0,True
150,common-2-3000,3000,4,False,-,1,True
151,grid-10-2,100,-1,True,-,2,True
152,gnm-4000-8000-0,4000,5,True,-,0,False


In [4]:
df.to_csv(r'results.txt', sep=' ', mode='a')

In [9]:
# comparing some results with hard-coded answers to see if they are correct
# comparing with all figures in red-scare pdf as we can see them with our EYES
correct_data = dict()
correct_data['Instance name'] = ['G-ex', 'P3', 'rusty-1-17', 'grid-5-0', 'wall-p-1', 'wall-p-3', 'wall-z-3', 'wall-n-2', 'ski-illustration', 'increase-n8-1', 'increase-n8-2']
# Len of shortest s-t path internally avoiding red nodes, -1 if no such path exists
correct_data['N'] = [3, -1, 10, 14, 1, 1, 1, 1, 8, 1, 1]
# True if path with 1 red otherwise False
correct_data['S'] = [True, True, True, True, True, True, False, False, '-', '-', '-']
# Max number of reds on a path from s to t, -1 if no path
correct_data['M'] = [2, 1, 1, 8, 1, 1, 0, 0, 1, 2, 1]
# Min number of reds on a path from s to t, -1 if no path
correct_data['F'] = [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
# True if alternate path exists, False otherwise
correct_data['A'] = [True, True, False, True, False, False, False, False, False, True, True]

correct_df = pd.DataFrame.from_dict(correct_data)
correct_df.head()

,Instance name,N,S,M,F,A
0,G-ex,3,True,2,0,True
1,P3,-1,True,1,1,True
2,rusty-1-17,10,True,1,0,False
3,grid-5-0,14,True,8,0,True
4,wall-p-1,1,True,1,0,False


In [10]:
print(correct_df['Instance name'].values)

['G-ex' 'P3' 'rusty-1-17' 'grid-5-0' 'wall-p-1' 'wall-p-3' 'wall-z-3'
 'wall-n-2' 'ski-illustration' 'increase-n8-1' 'increase-n8-2']


In [11]:
# compare results
for idx, row in df.iterrows():
    instance = row['Instance name']

    if instance in correct_df['Instance name'].values:
        correct_row = correct_df[correct_df['Instance name'] == instance]

        # NONE
        if row['N'] != correct_row['N'].values[0]:
            print(f'===== Issue with None with instance {instance}! =====')
            print(f'our answer {row["N"]} ===== correct answer {correct_row["N"].values[0]}\n')

        # SOME (uncomment below when it is added)
        if row['S'] != correct_row['S'].values[0]:
            print(f'===== Issue with Some with instance {instance}! =====')
            print(f'our answer {row["S"]} ===== correct answer {correct_row["S"].values[0]}\n')

        # MANY
        if row['M'] != correct_row['M'].values[0] and row['M'] != '-':
            print(f'===== Issue with Many with instance {instance}! =====')
            print(f'our answer {row["M"]} ===== correct answer {correct_row["M"].values[0]}\n')

        # FEW
        if row['F'] != correct_row['F'].values[0]:
            print(f'===== Issue with Few with instance {instance}! =====')
            print(f'our answer {row["F"]} ===== correct answer {correct_row["F"].values[0]}\n')

        # ALTERNATE
        if row['A'] != correct_row['A'].values[0]:
            print(f'===== Issue with Alternate with instance {instance}! =====')
            print(f'our answer {row["A"]} ===== correct answer {correct_row["A"].values[0]}\n')

In [8]:
### Running example:
g, s, t = create_network('../data/wall-p-10000.txt')
for i in [solve_none, solve_some, solve_many, solve_few, solve_alternate]:
    start = time.time()
    solution = i(g, s, t)
    print(str(i), round(time.time() - start, 4), solution)

<function solve_none at 0x132d977e0> 0.5609 1
<function solve_some at 0x13312d4e0> 279.3034 True
<function solve_many at 0x13312dda0> 0.1114 -
<function solve_few at 0x13312df80> 0.0887 0
<function solve_alternate at 0x13312e160> 0.1228 False
